In [1]:
import pandas as pd
import wbgapi as wb

### Needed Series from WDI Database 
- EN.ATM.CO2E.KT - CO2 emissions (kt)
- EN.ATM.CO2E.PC - CO2 emissions (metric tons per capita)
- EN.ATM.CO2E.PP.GD.KD	- CO2 emissions (kg per 2017 PPP dollar of GDP)
- EG.USE.PCAP.KG.OE	 - Energy Use (kg of oil equivalent per capita)
- EG.FEC.RNEW.ZS - Renewable energy consumption (% of total final energy consumption)
- AG.YLD.CREL.KG - Cereal yield
- EG.ELC.ACCS.ZS - Access to Electricity (% of population)
- AG.LND.FRST.ZS - Forest area (% of land area)
- NY.GDP.MKTP.KD.ZG - GDP growth (annual %)
- NY.GDP.PCAP.CD - GDP per capita (current USD)
- NY.GNP.PCAP.CD - GNI per capita, Atlas method (current USD)
- SP.POP.TOTL - Total Population
- SP.POP.GROW - Population growth (annual %)
- SP.URB.GROW - Urban population growth (annual %)
- SP.URB.TOTL - Urban population
- SP.URB.TOTL.IN.ZS	- Urban population (% of total population)

## Extract emissions data from World Bank database via API (wbgapi)
### Ensure conda environment has wbgapi installed (pip install wbgapi)

In [2]:
series_list = ['EN.ATM.CO2E.KT',
               'EN.ATM.CO2E.PC',
               'EN.ATM.CO2E.PP.GD.KD',
               'EG.USE.PCAP.KG.OE',
               'EG.FEC.RNEW.ZS',
               'AG.YLD.CREL.KG',
               'EG.ELC.ACCS.ZS',
               'AG.LND.FRST.ZS',
               'NY.GDP.MKTP.KD.ZG',
               'NY.GDP.PCAP.CD',
               'NY.GNP.PCAP.CD',
               'SP.POP.TOTL',
               'SP.POP.GROW',
               'SP.URB.GROW',
               'SP.URB.TOTL',
               'SP.URB.TOTL.IN.ZS']

In [3]:
# Using wbgapi to extract World Bank data as Pandas data frame
raw_df = wb.data.DataFrame(series_list, time=range(1990, 2018), numericTimeKeys=True, labels=True, columns='series').reset_index()
raw_df.head()

,economy,time,Country,Time,AG.LND.FRST.ZS,AG.YLD.CREL.KG,EG.ELC.ACCS.ZS,EG.FEC.RNEW.ZS,EG.USE.PCAP.KG.OE,EN.ATM.CO2E.KT,EN.ATM.CO2E.PC,EN.ATM.CO2E.PP.GD.KD,NY.GDP.MKTP.KD.ZG,NY.GDP.PCAP.CD,NY.GNP.PCAP.CD,SP.POP.GROW,SP.POP.TOTL,SP.URB.GROW,SP.URB.TOTL,SP.URB.TOTL.IN.ZS
0,ZWE,2017,Zimbabwe,2017,45.451183,1202.7,44.178635,82.46,NaN,10340.000153,0.700965,0.300613,4.080264,1192.107012,1170.0,2.043620,14751101.0,1.860765,4755312.0,32.237
1,ZWE,2016,Zimbabwe,2016,45.570273,435.1,42.561729,81.90,NaN,11020.000458,0.762487,0.333455,0.900955,1421.787789,1200.0,2.081806,14452704.0,1.806610,4667645.0,32.296
2,ZWE,2015,Zimbabwe,2015,45.689363,557.5,33.700001,80.82,NaN,12430.000305,0.878139,0.379509,2.023650,1410.329174,1220.0,2.136294,14154937.0,1.769505,4584076.0,32.385
3,ZWE,2014,Zimbabwe,2014,45.808453,831.4,32.299999,80.27,NaN,12079.999924,0.871840,0.376287,1.484543,1407.034293,1210.0,2.191391,13855753.0,1.730983,4503674.0,32.504
4,ZWE,2013,Zimbabwe,2013,45.927543,668.5,40.498375,78.87,832.572236,12279.999733,0.905911,0.388196,3.196731,1408.367810,1200.0,2.163267,13555422.0,1.613531,4426387.0,32.654


In [4]:
# Rows and columns of data set
raw_df.shape

(7448, 20)

In [5]:
# # Datatypes of columns
raw_df.dtypes

economy                  object
time                      int64
Country                  object
Time                     object
AG.LND.FRST.ZS          float64
AG.YLD.CREL.KG          float64
EG.ELC.ACCS.ZS          float64
EG.FEC.RNEW.ZS          float64
EG.USE.PCAP.KG.OE       float64
EN.ATM.CO2E.KT          float64
EN.ATM.CO2E.PC          float64
EN.ATM.CO2E.PP.GD.KD    float64
NY.GDP.MKTP.KD.ZG       float64
NY.GDP.PCAP.CD          float64
NY.GNP.PCAP.CD          float64
SP.POP.GROW             float64
SP.POP.TOTL             float64
SP.URB.GROW             float64
SP.URB.TOTL             float64
SP.URB.TOTL.IN.ZS       float64
dtype: object

In [6]:
# Descriptive statistics
raw_df.describe()

,time,AG.LND.FRST.ZS,AG.YLD.CREL.KG,EG.ELC.ACCS.ZS,EG.FEC.RNEW.ZS,EG.USE.PCAP.KG.OE,EN.ATM.CO2E.KT,EN.ATM.CO2E.PC,EN.ATM.CO2E.PP.GD.KD,NY.GDP.MKTP.KD.ZG,NY.GDP.PCAP.CD,NY.GNP.PCAP.CD,SP.POP.GROW,SP.POP.TOTL,SP.URB.GROW,SP.URB.TOTL,SP.URB.TOTL.IN.ZS
count,7448.00000,7117.000000,6175.000000,6305.000000,7089.000000,4751.000000,6.669000e+03,6669.000000,6206.000000,6782.000000,6944.000000,6393.000000,7418.000000,7.420000e+03,7363.000000,7.364000e+03,7364.000000
mean,2003.50000,32.625391,2894.700192,79.933424,31.159401,2270.160374,9.927095e+05,4.210340,0.268540,3.581076,11171.752248,9479.314154,1.478696,2.583084e+08,2.226490,1.210294e+08,55.266415
std,8.07829,23.388023,2336.051682,29.463801,30.118135,2669.532102,3.204385e+06,5.229999,0.215092,5.787170,18975.239473,15075.519703,1.650633,8.196081e+08,2.092501,3.806065e+08,23.529375
min,1990.00000,0.000000,0.100000,0.533899,0.000000,9.579196,0.000000e+00,0.000000,0.000000,-64.047107,22.850371,40.000000,-27.722225,9.182000e+03,-27.707932,3.733000e+03,5.416000
25%,1996.75000,12.513950,1404.050000,65.926689,4.785963,603.049194,2.230000e+03,0.639336,0.138454,1.545476,1007.129242,910.000000,0.546640,1.330466e+06,0.774823,6.515722e+05,35.295750
50%,2003.50000,30.855176,2388.700000,98.300003,20.946516,1238.114597,2.374000e+04,2.421594,0.215263,3.712218,3322.033110,3020.000000,1.408416,8.483160e+06,2.199913,4.041233e+06,54.165000
75%,2010.25000,47.617367,3796.250000,100.000000,54.760000,3025.736971,2.464900e+05,6.218103,0.330033,5.902814,13096.144542,10400.000000,2.416364,5.593234e+07,3.503700,3.182758e+07,74.308513
max,2017.00000,98.574551,36761.900000,100.000000,98.340000,21420.628504,3.351454e+07,47.651306,2.085052,149.972963,203266.913745,122130.000000,19.360429,7.578158e+09,31.143425,4.147419e+09,100.000000


## Data clean and prep

In [9]:
# Assign original dataframe to another that can be modified
emissions_df = raw_df
emissions_df.sample(5)

,economy,time,Country,Time,AG.LND.FRST.ZS,AG.YLD.CREL.KG,EG.ELC.ACCS.ZS,EG.FEC.RNEW.ZS,EG.USE.PCAP.KG.OE,EN.ATM.CO2E.KT,EN.ATM.CO2E.PC,EN.ATM.CO2E.PP.GD.KD,NY.GDP.MKTP.KD.ZG,NY.GDP.PCAP.CD,NY.GNP.PCAP.CD,SP.POP.GROW,SP.POP.TOTL,SP.URB.GROW,SP.URB.TOTL,SP.URB.TOTL.IN.ZS
1659,PRT,2010,Portugal,2010,35.506060,3351.300000,100.000000,27.830000,2222.629692,50939.998627,4.817887,0.151515,1.737625,22520.642312,23030.000000,0.045910,10573100.0,1.046478,6403809.0,60.567000
7020,HPC,1997,Heavily indebted poor countries (HIPC),1997,30.505149,1014.548310,NaN,86.329242,NaN,72309.738053,0.165196,0.092688,6.200500,349.130650,326.907644,2.790146,437721228.0,3.860460,121696838.0,27.802361
782,SYR,1991,Syrian Arab Republic,1991,2.057242,922.400000,NaN,2.107307,892.028426,35770.000000,2.798405,NaN,7.666468,2171.463634,NaN,2.963822,12782281.0,3.388006,6281085.0,49.139000
4740,CIV,2009,Cote d'Ivoire,2009,12.825368,1711.500000,57.315105,73.710000,453.477992,6030.000210,0.291618,0.083051,3.603322,1638.804766,1150.000000,2.117819,20677762.0,3.015028,9699938.0,46.910000
6541,TMN,2000,Middle East & North Africa (IDA & IBRD countries),2000,2.245404,1770.512593,91.372682,2.945707,1001.590501,812780.000000,2.892693,0.363702,6.197073,1878.091589,1747.982853,1.984886,280976957.0,2.664402,154878632.0,55.121471


In [10]:
emissions_df.isnull().sum()

economy                    0
time                       0
Country                    0
Time                       0
AG.LND.FRST.ZS           331
AG.YLD.CREL.KG          1273
EG.ELC.ACCS.ZS          1143
EG.FEC.RNEW.ZS           359
EG.USE.PCAP.KG.OE       2697
EN.ATM.CO2E.KT           779
EN.ATM.CO2E.PC           779
EN.ATM.CO2E.PP.GD.KD    1242
NY.GDP.MKTP.KD.ZG        666
NY.GDP.PCAP.CD           504
NY.GNP.PCAP.CD          1055
SP.POP.GROW               30
SP.POP.TOTL               28
SP.URB.GROW               85
SP.URB.TOTL               84
SP.URB.TOTL.IN.ZS         84
dtype: int64

In [11]:
# Rename the columns headers to meaningful names
column_names = {'AG.LND.FRST.ZS':'forest_area_percent',
             'AG.YLD.CREL.KG':'cereal_yield',
             'EG.ELC.ACCS.ZS':'electricity_access_percent',
             'EG.FEC.RNEW.ZS':'renew_energy_percent',
             'EG.USE.PCAP.KG.OE':'energy_use_per_capita',
             'EN.ATM.CO2E.KT':'emissions_total',
             'EN.ATM.CO2E.PC':'emissions_per_capita',
             'EN.ATM.CO2E.PP.GD.KD':'emissions_per_gdp',
             'NY.GDP.MKTP.KD.ZG':'gdp_growth_percent',
             'NY.GDP.PCAP.CD':'gdp_per_capita',
             'NY.GNP.PCAP.CD':'gni_per_capita',
             'SP.POP.GROW':'pop_growth_percent',
             'SP.POP.TOTL':'pop_total',
             'SP.URB.GROW':'urb_pop_growth_percent',
             'SP.URB.TOTL':'urban_pop_total',
             'SP.URB.TOTL.IN.ZS':'urban_pop_percent'
            }

In [12]:
emissions_df = emissions_df.rename(columns=column_names)
emissions_df.sample(10)

,economy,time,Country,Time,forest_area_percent,cereal_yield,electricity_access_percent,renew_energy_percent,energy_use_per_capita,emissions_total,emissions_per_capita,emissions_per_gdp,gdp_growth_percent,gdp_per_capita,gni_per_capita,pop_growth_percent,pop_total,urb_pop_growth_percent,urban_pop_total,urban_pop_percent
4440,EGY,2001,"Egypt, Arab Rep.",2001,0.060127,7099.400000,97.344078,7.820000,625.647744,1.267000e+05,1.739088,0.229560,3.535252,1327.096518,1380.000000,2.056420,7.285426e+07,2.163845,3.121295e+07,42.843000
3326,JPN,1995,Japan,1995,68.329676,6003.200000,100.000000,3.917189,3934.959561,1.171010e+06,9.332839,0.267665,2.631000,44197.619101,42570.000000,0.234590,1.254720e+08,0.407782,9.788824e+07,78.016000
3452,IRL,2009,Ireland,2009,10.326288,6887.200000,100.000000,5.300000,3149.565141,4.056000e+04,8.943031,0.169046,-5.095783,52133.090616,46810.000000,1.015663,4.535375e+06,1.343886,2.782090e+06,61.342000
4514,DMA,2011,Dominica,2011,63.826667,1400.000000,95.536674,13.800000,NaN,1.800000e+02,2.618487,0.212715,-0.223551,7288.497948,7140.000000,-0.018910,6.874200e+04,0.413514,4.701200e+04,68.389000
7225,EAP,2016,East Asia & Pacific (excluding high income),2016,30.282198,5361.116973,96.904829,NaN,NaN,1.130884e+07,5.475843,0.446529,6.585886,6591.282343,6652.983041,0.751984,2.065223e+09,2.655708,1.109067e+09,53.702045
1816,PNG,1993,Papua New Guinea,1993,80.296705,2680.000000,NaN,71.249161,NaN,2.210000e+03,0.514961,0.133411,18.202286,1159.139760,1130.000000,3.677200,4.291588e+06,2.411408,6.199200e+05,14.445000
4995,TCD,2006,Chad,2006,4.653113,766.800000,5.196557,83.740000,NaN,5.100000e+02,0.049201,0.031015,0.648262,716.667803,540.000000,3.540781,1.036561e+07,3.692014,2.263228e+06,21.834000
1879,PAK,2014,Pakistan,2014,5.143247,3001.200000,71.224739,47.220000,431.626863,1.542400e+05,0.740642,0.168187,4.674708,1173.392454,1160.000000,1.409183,2.082516e+08,1.985957,7.459365e+07,35.819000
419,UGA,1990,Uganda,1990,17.894350,1497.600000,NaN,96.018513,NaN,7.900000e+02,0.044920,0.049820,6.474140,244.754047,320.000000,3.224802,1.758663e+07,7.006126,1.947895e+06,11.076000
6763,LAC,2002,Latin America & Caribbean (excluding high income),2002,51.404326,2899.918782,92.062502,29.800250,1059.568185,1.037720e+06,2.152437,0.182599,0.428243,3568.863649,3640.149698,1.374544,4.821140e+08,1.856167,3.623179e+08,75.151906
